In [1]:
import fml_lib
import numpy as np
import pandas as pd
import scipy.stats as ss
import os.path
from math import sqrt, exp, log
import matplotlib.pyplot as plt

In [2]:
dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])
dollar_bars = dollar_bars.set_index("Date_Timestamp")
close = dollar_bars['Close']
daily_volatility = fml_lib.getDailyVol(dollar_bars.Close,span0=100)


C:\Users\dstek\AppData\Local\Temp\ipykernel_30648\1907931566.py:1: FutureWarning: Support for nested sequences for 'parse_dates' in pd.read_csv is deprecated. Combine the desired columns with pd.to_datetime after parsing instead.
  dollar_bars = pd.read_csv("USA500IDXUSD_bars.csv", parse_dates=[['Date', 'Timestamp']])


In [3]:

if os.path.exists("./t_events.csv"):
    t_events = pd.read_csv("t_events.csv", index_col=[0])
    t_events = t_events.index
else:
    t_events = fml_lib.getTEvents(dollar_bars.Close, daily_volatility)
    t_events_df = pd.DataFrame(index= t_events)
    t_events_df.to_csv("./t_events_csv", index=True)

In [4]:
t_events = pd.to_datetime(t_events)
print(t_events)

DatetimeIndex(['2023-07-04 22:00:10', '2023-07-04 22:00:40',
               '2023-07-04 22:00:50', '2023-07-04 22:01:10',
               '2023-07-04 22:01:30', '2023-07-04 22:01:50',
               '2023-07-04 22:02:10', '2023-07-04 22:02:20',
               '2023-07-04 22:02:40', '2023-07-04 22:03:00',
               ...
               '2024-01-25 03:52:20', '2024-01-25 03:53:40',
               '2024-01-25 03:54:50', '2024-01-25 03:55:10',
               '2024-01-25 03:55:40', '2024-01-25 03:56:50',
               '2024-01-25 03:57:00', '2024-01-25 03:57:20',
               '2024-01-25 03:58:30', '2024-01-25 03:58:40'],
              dtype='datetime64[ns]', length=814094, freq=None)


In [5]:


#adds the vertical barrier num_days out from the time of the event
numDays = 10
t1=close.index.searchsorted(t_events+pd.Timedelta(days=numDays))
t1=t1[t1<close.shape[0]]
t1=pd.Series(close.index[t1],index=t_events[:t1.shape[0]]) # NaNs at end
print(t1)

2023-07-04 22:00:10   2023-07-16 22:00:00
2023-07-04 22:00:40   2023-07-16 22:00:00
2023-07-04 22:00:50   2023-07-16 22:00:00
2023-07-04 22:01:10   2023-07-16 22:00:00
2023-07-04 22:01:30   2023-07-16 22:00:00
                              ...        
2024-01-15 03:54:40   2024-01-25 03:54:50
2024-01-15 03:56:00   2024-01-25 03:56:50
2024-01-15 03:57:00   2024-01-25 03:57:00
2024-01-15 03:57:30   2024-01-25 03:58:30
2024-01-15 03:58:40   2024-01-25 03:58:40
Name: Date_Timestamp, Length: 769572, dtype: datetime64[ns]


In [6]:
min_ret= 0.005
trgt = pd.Series(0.01, index=t_events)
if os.path.exists("./events.csv"):
    events = pd.read_csv("events.csv", index_col=[0])
    events = events.dropna()
else:
    events = fml_lib.getEvents(close, t_events, 0.02,trgt ,min_ret,t1)
    events.to_csv("events.csv")
print(events)

                                      t1  trgt
2023-07-04 22:00:10  2023-07-04 22:17:50  0.01
2023-07-04 22:00:40  2023-07-04 22:17:50  0.01
2023-07-04 22:00:50  2023-07-04 22:15:30  0.01
2023-07-04 22:01:10  2023-07-04 22:17:50  0.01
2023-07-04 22:01:30  2023-07-04 22:17:50  0.01
...                                  ...   ...
2024-01-25 03:50:30  2024-01-25 03:56:50  0.01
2024-01-25 03:51:30  2024-01-25 03:54:50  0.01
2024-01-25 03:51:40  2024-01-25 03:56:50  0.01
2024-01-25 03:51:50  2024-01-25 03:54:50  0.01
2024-01-25 03:52:00  2024-01-25 03:56:50  0.01

[814081 rows x 2 columns]


In [7]:

bins = fml_lib.getBins(events, close, numDays)

[ True  True  True ...  True  True  True]


In [8]:
print(bins[bins['bin'] == -1])
print(bins)

                          ret  bin
2023-07-04 22:02:10 -0.000225 -1.0
2023-07-04 22:02:40 -0.000223 -1.0
2023-07-04 22:03:00 -0.000223 -1.0
2023-07-04 22:03:10 -0.000210 -1.0
2023-07-04 22:13:00 -0.000222 -1.0
...                       ...  ...
2024-01-25 03:40:30 -0.000205 -1.0
2024-01-25 03:40:40 -0.000207 -1.0
2024-01-25 03:40:50 -0.000210 -1.0
2024-01-25 03:41:10 -0.000207 -1.0
2024-01-25 03:41:40 -0.000209 -1.0

[401614 rows x 2 columns]
                          ret  bin
2023-07-04 22:00:10  0.000226  1.0
2023-07-04 22:00:40  0.000223  1.0
2023-07-04 22:00:50  0.000220  1.0
2023-07-04 22:01:10  0.000226  1.0
2023-07-04 22:01:30  0.000224  1.0
...                       ...  ...
2024-01-25 03:50:30  0.000207  1.0
2024-01-25 03:51:30  0.000216  1.0
2024-01-25 03:51:40  0.000208  1.0
2024-01-25 03:51:50  0.000215  1.0
2024-01-25 03:52:00  0.000207  1.0

[814081 rows x 2 columns]


Want to find sample weights for use in the boosting fit

In [9]:
co_events = fml_lib.get_co_events(close,events)
print(co_events)

ValueError: No objects to concatenate